In [1]:
from matplotlib import pyplot as plt
from scipy import stats
import numpy as np
import pandas as pd
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo
import pymongo
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import time 
from IPython import get_ipython
import nbconvert

In [2]:
#Import CSVs of individual WR statistics for years 2009-2019 (source: airyards.com)

ay_2019 = pd.read_csv("../Resources/2019.csv")
ay_2018 = pd.read_csv("../Resources/2018.csv")
ay_2017 = pd.read_csv("../Resources/2017.csv")
ay_2016 = pd.read_csv("../Resources/2016.csv")
ay_2015 = pd.read_csv("../Resources/2015.csv")
ay_2014 = pd.read_csv("../Resources/2014.csv")
ay_2013 = pd.read_csv("../Resources/2013.csv")
ay_2012 = pd.read_csv("../Resources/2012.csv")
ay_2011 = pd.read_csv("../Resources/2011.csv")
ay_2010 = pd.read_csv("../Resources/2010.csv")
ay_2009 = pd.read_csv("../Resources/2009.csv")


# create list of objects for read results
df_list = [ay_2009, ay_2010, ay_2011, ay_2012, ay_2013, ay_2014, ay_2015, ay_2016, ay_2017, ay_2018, ay_2019]


df_list

[    Unnamed: 0            full_name position team  targets  rec  rec_yards  \
 0            1     Larry Fitzgerald       WR  ARI      154   97       1092   
 1            2     Brandon Marshall       WR  DEN      154  101       1120   
 2            3       DeSean Jackson       WR  PHI      118   63       1167   
 3            4         Mike Wallace       WR  PIT       72   39        756   
 4            5        Jeremy Maclin       WR  PHI       90   55        762   
 5            6        Pierre Garcon       WR  IND       92   47        765   
 6            7          Kenny Britt       WR  TEN       75   42        701   
 7            8     Michael Crabtree       WR   SF       86   48        625   
 8            9             Ted Ginn       WR  MIA       78   38        454   
 9           10       Danny Amendola       WR  STL       63   43        326   
 10          11       Julian Edelman       WR   NE       54   37        359   
 11          12         Jordy Nelson       WR   GB  

In [3]:
#Add Year column

year = 2009

for df in df_list:
    
    df["Year"] = year
    
    year += 1

df_list[10]

Unnamed: 0        full_name position team  targets  rec  rec_yards  \
0             1   Michael Thomas       WR   NO      185  149       1725   
1             2     Chris Godwin       WR   TB      121   86       1333   
2             3      Julio Jones       WR  ATL      157   99       1394   
3             4      Cooper Kupp       WR   LA      134   94       1161   
4             5  DeAndre Hopkins       WR  HOU      150  104       1165   
..          ...              ...      ...  ...      ...  ...        ...   
198         199   Juwann Winfree       WR  DEN        0    0          0   
199         200     Mack Hollins       WR  MIA        1    0          0   
200         201   Matthew Slater       WR   NE        0    0          0   
201         202    Taywan Taylor       WR  CLE        1    0          0   
202         203     Trevor Davis       WR  MIA        0    0          0   

     air_yards  yac  td  adot  racr  ms_air  tgt_share  wopr    ppr  Year  
0         1488  583   9   8.0  1.16    0.40       0.32  0.75  374.6  2019  
1         1301  577   9  10.8  1.02    0.23       0.22  0.49  274.1  2019  
2         1913  364   6  12.2  0.73    0.37       0.25  0.63  274.1  2019  
3         1007  538  10   7.5  1.15    0.21       0.21  0.46  270.5  2019  
4         1580  387   7  10.5  0.74    0.34       0.30  0.69  264.3  2019  
..         ...  ...  ..   ...   ...     ...        ...   ...    ...   ...  
198          0    0   0   NaN   NaN    0.00       0.00  0.00    0.0  2019  
199         33    0   0  33.0  0.00    0.03       0.01  0.03    0.0  2019  
200          0    0   0   NaN   NaN    0.00       0.00  0.00    0.0  2019  
201         33    0   0  33.0  0.00    0.05       0.02  0.06    0.0  2019  
202          0    0   0   NaN   NaN    0.00       0.00  0.00    0.0  2019  

[203 rows x 17 columns]

In [4]:
merged_df = pd.concat(df_list)


In [5]:
merged_df

Unnamed: 0         full_name position team  targets  rec  rec_yards  \
0             1  Larry Fitzgerald       WR  ARI      154   97       1092   
1             2  Brandon Marshall       WR  DEN      154  101       1120   
2             3    DeSean Jackson       WR  PHI      118   63       1167   
3             4      Mike Wallace       WR  PIT       72   39        756   
4             5     Jeremy Maclin       WR  PHI       90   55        762   
..          ...               ...      ...  ...      ...  ...        ...   
198         199    Juwann Winfree       WR  DEN        0    0          0   
199         200      Mack Hollins       WR  MIA        1    0          0   
200         201    Matthew Slater       WR   NE        0    0          0   
201         202     Taywan Taylor       WR  CLE        1    0          0   
202         203      Trevor Davis       WR  MIA        0    0          0   

     air_yards  yac  td  adot  racr  ms_air  tgt_share  wopr    ppr  Year  
0         1779  312  13  11.6  0.61    0.39       0.26  0.66  284.2  2009  
1         1367  515  10   8.9  0.82    0.40       0.31  0.74  276.9  2009  
2         1599  404   9  13.6  0.73    0.36       0.22  0.59  253.4  2009  
3         1359  143   6  18.9  0.56    0.27       0.13  0.39  155.4  2009  
4         1057  241   4  11.7  0.72    0.27       0.20  0.48  154.5  2009  
..         ...  ...  ..   ...   ...     ...        ...   ...    ...   ...  
198          0    0   0   NaN   NaN    0.00       0.00  0.00    0.0  2019  
199         33    0   0  33.0  0.00    0.03       0.01  0.03    0.0  2019  
200          0    0   0   NaN   NaN    0.00       0.00  0.00    0.0  2019  
201         33    0   0  33.0  0.00    0.05       0.02  0.06    0.0  2019  
202          0    0   0   NaN   NaN    0.00       0.00  0.00    0.0  2019  

[1103 rows x 17 columns]

In [6]:
# add prior year stats

merged_df = merged_df[['full_name', 'Year', 'targets', 'rec', 'rec_yards', 'air_yards', 'yac', 'td', 'adot', 'racr', 'ms_air',
                     'tgt_share', 'wopr', 'ppr']]

merged_df = merged_df.sort_values(by=['full_name', 'Year'], ascending=False)

merged_df.head(20)


full_name  Year  targets  rec  rec_yards  air_yards  yac  td  adot  \
129      Zay Jones  2019       27   20        147        234   58   0   8.7   
152      Zay Jones  2019       18    7         69        250   19   0  13.9   
37       Zay Jones  2018      100   54        621       1256  133   6  12.6   
78       Zay Jones  2017       74   27        316        994   63   2  13.4   
53     Zach Pascal  2019       72   41        607        800  235   5  11.1   
92     Zach Pascal  2018       44   26        272        470   71   2  10.7   
73    Willie Snead  2019       46   31        339        409  126   5   8.9   
44    Willie Snead  2018       95   62        649        775  276   1   8.2   
121   Willie Snead  2017       16    8         92        107   32   0   6.7   
33    Willie Snead  2016      104   72        895        783  387   4   7.5   
28    Willie Snead  2015      102   70        990       1032  344   3  10.1   
51     Will Fuller  2019       71   49        670       1011  219   3  14.2   
63     Will Fuller  2018       45   32        503        623  166   4  13.8   
55     Will Fuller  2017       50   28        423        794   99   7  15.9   
61     Will Fuller  2016       92   47        635       1428  238   2  15.5   
118    Walt Powell  2016       25   14        142        302   45   0  12.1   
125    Walt Powell  2015        0    0          0          0    0   0   NaN   
93     Walt Powell  2014        0    0          0          0    0   0   NaN   
107  Vyncint Smith  2019       31   17        225        378  105   0  12.2   
163    Vince Mayle  2016        0    0          0          0    0   0   NaN   

     racr  ms_air  tgt_share  wopr    ppr  
129  0.63    0.11       0.08  0.20   35.0  
152  0.28    0.18       0.12  0.31   13.9  
37   0.49    0.25       0.21  0.48  152.1  
78   0.32    0.27       0.17  0.44   70.6  
53   0.76    0.19       0.14  0.35  133.3  
92   0.58    0.11       0.08  0.19   66.2  
73   0.83    0.11       0.10  0.23   95.1  
44   0.84    0.17       0.18  0.39  134.2  
121  0.86    0.07       0.06  0.14   17.2  
33   1.14    0.18       0.17  0.38  185.5  
28   0.96    0.22       0.16  0.40  187.0  
51   0.66    0.29       0.20  0.51  134.0  
63   0.81    0.29       0.20  0.50  106.3  
55   0.53    0.24       0.16  0.40  113.2  
61   0.44    0.32       0.18  0.49  122.2  
118  0.47    0.15       0.12  0.28   28.2  
125   NaN    0.00       0.00  0.00    0.0  
93    NaN    0.00       0.00  0.00    0.0  
107  0.60    0.11       0.07  0.18   50.7  
163   NaN    0.00       0.00  0.00    0.0

In [7]:
merged_df['UAY'] = merged_df['air_yards'] - (merged_df['rec_yards'] - merged_df['yac'])

merged_df['Catch %'] = (merged_df['rec']/merged_df['targets']) * 100

merged_df['Yds/tgt'] = (merged_df['rec_yards']/merged_df['targets'])

merged_df['Yds/rec'] = (merged_df['rec_yards']/merged_df['rec'])

merged_df['AY/rec'] = (merged_df['air_yards']/merged_df['rec'])

merged_df['AY/tgt'] = (merged_df['air_yards']/merged_df['targets'])

merged_df['UAY/tgt'] = (merged_df['UAY']/merged_df['targets'])

merged_df['UAY/rec'] = (merged_df['UAY']/merged_df['rec'])

merged_df['YAC/tgt'] = (merged_df['yac']/merged_df['targets'])

merged_df['YAC/rec'] = (merged_df['yac']/merged_df['rec'])

merged_df.head(50)

full_name  Year  targets  rec  rec_yards  air_yards  yac  td  adot  \
129        Zay Jones  2019       27   20        147        234   58   0   8.7   
152        Zay Jones  2019       18    7         69        250   19   0  13.9   
37         Zay Jones  2018      100   54        621       1256  133   6  12.6   
78         Zay Jones  2017       74   27        316        994   63   2  13.4   
53       Zach Pascal  2019       72   41        607        800  235   5  11.1   
92       Zach Pascal  2018       44   26        272        470   71   2  10.7   
73      Willie Snead  2019       46   31        339        409  126   5   8.9   
44      Willie Snead  2018       95   62        649        775  276   1   8.2   
121     Willie Snead  2017       16    8         92        107   32   0   6.7   
33      Willie Snead  2016      104   72        895        783  387   4   7.5   
28      Willie Snead  2015      102   70        990       1032  344   3  10.1   
51       Will Fuller  2019       71   49        670       1011  219   3  14.2   
63       Will Fuller  2018       45   32        503        623  166   4  13.8   
55       Will Fuller  2017       50   28        423        794   99   7  15.9   
61       Will Fuller  2016       92   47        635       1428  238   2  15.5   
118      Walt Powell  2016       25   14        142        302   45   0  12.1   
125      Walt Powell  2015        0    0          0          0    0   0   NaN   
93       Walt Powell  2014        0    0          0          0    0   0   NaN   
107    Vyncint Smith  2019       31   17        225        378  105   0  12.2   
163      Vince Mayle  2016        0    0          0          0    0   0   NaN   
124      Vince Mayle  2015        1    0          0         23    0   0  23.0   
153    Victor Bolden  2017        0    0          0          0    0   0   NaN   
168   Ventell Bryant  2019        1    1         15         15    0   1  15.0   
45   Tyrell Williams  2019       64   42        651        870  190   6  13.6   
43   Tyrell Williams  2018       63   40        631        789  183   5  12.5   
41   Tyrell Williams  2017       69   43        728        988  320   4  14.3   
17   Tyrell Williams  2016      119   69       1059       1382  433   7  11.6   
103  Tyrell Williams  2015        6    2         90        161   51   1  26.8   
31       Tyreek Hill  2019       89   58        860       1149  275   7  12.9   
6        Tyreek Hill  2018      132   82       1336       1898  520  10  14.4   
8        Tyreek Hill  2017      105   75       1183       1227  451   7  11.7   
21       Tyreek Hill  2016       83   61        593        662  268   6   8.0   
12     Tyler Lockett  2019      110   82       1057       1364  308   8  12.4   
16     Tyler Lockett  2018       67   55        970        910  227  10  13.6   
52     Tyler Lockett  2017       71   45        555        922  188   2  13.0   
60     Tyler Lockett  2016       66   41        597        698  216   1  10.6   
38     Tyler Lockett  2015       69   51        664        923  249   6  13.4   
16        Tyler Boyd  2019      148   90       1046       1186  423   5   8.0   
14        Tyler Boyd  2018      108   76       1028       1087  426   7  10.1   
89        Tyler Boyd  2017       32   22        225        218   91   2   6.8   
58        Tyler Boyd  2016       81   54        603        674  201   1   8.3   
105       Trey Quinn  2019       47   26        198        324   61   1   6.9   
131       Trey Quinn  2018       10    9         75         73   17   1   7.3   
136     Trevor Davis  2019       10    7         83         82   55   0   8.2   
186     Trevor Davis  2019        1    1         28         18   10   0  18.0   
202     Trevor Davis  2019        0    0          0          0    0   0   NaN   
166     Trevor Davis  2018        0    0          0          0    0   0   NaN   
130     Trevor Davis  2017        7    5         70         69   34   0   9.9   
139     Trevor Davis  2016        7    3  

In [8]:
for col in merged_df.columns: 
    print(col)

full_name
Year
targets
rec
rec_yards
air_yards
yac
td
adot
racr
ms_air
tgt_share
wopr
ppr
UAY
Catch %
Yds/tgt
Yds/rec
AY/rec
AY/tgt
UAY/tgt
UAY/rec
YAC/tgt
YAC/rec


In [9]:
name_list = []

name_list2 = []

# Current Year Lists

yearList = []
tgtList = []
recList = []
recYdsList = []
ayList = []
yacList = []
tdList = []
adotList = []
racrList = []
msAirList = []
tgtShareList = []
woprList = []
pprPointsList = []
uayList = []
catchRateList = []
yptList = []
yprList = []
ayPerRecList = []
ayPerTgtList = []
uayPerTgtList = []
uayPerRecList = []
yacPerTgtList = []
yacPerRecList = []


# Prior Years Lists (py = Prior Year)

py_ayList = []
py_tarList = []
py_recList = []
py_ydsList = []
py_yacList = []
py_tdList = []
py_adotList = []
py_racrList = []

py_msAirList = []
py_tgtShare = []
py_woprList = []
py_pprPointsList = []
py_uayList = []
py_catchRateList = []
py_yptList = []
py_yprList = []
py_ayPerRecList = []
py_ayPerTgtList = []
py_uayPerTgtList = []
py_uayPerRecList = []
py_yacPerTgtList = []
py_yacPerRecList = []

for i, row in merged_df.iterrows():
    
    name = row[0]
    year = row[1]
    tar = row[2]
    rec = row[3]
    yds = row[4]
    AY = row[5]
    yac = row[6]
    td = row[7]
    adot = row[19]
    racr = row[9]
    ms_air = row[10]
    tgt_share = row[11]
    wopr = row[12]
    ppr = row[13]
    UAY = row[14]
    catch_rate = row[15]
    ypt = row[16]
    ypr = row[17]
    ayrec = row[18]
    aytar = row[19]
    uaytar = row[20]
    uayrec = row[21]
    yactgt = row[22]
    yacrec = row[23]
    
    if name not in name_list:
        
        name_list.append(name)
        name_list2.append(name)
        yearList.append(year)
        
        tgtList.append(tar)
        recList.append(rec)
        recYdsList.append(yds)
        ayList.append(AY)
        yacList.append(yac)
        tdList.append(td)
        adotList.append(adot)
        racrList.append(racr)
        msAirList.append(ms_air)
        tgtShareList.append(tgt_share)
        woprList.append(wopr)
        pprPointsList.append(ppr)
        uayList.append(UAY)
        catchRateList.append(catch_rate)
        yptList.append(ypt)
        yprList.append(ypr)
        ayPerRecList.append(ayrec)
        ayPerTgtList.append(aytar)
        uayPerTgtList.append(uaytar)
        uayPerRecList.append(uayrec)
        yacPerTgtList.append(yactgt)
        yacPerRecList.append(yacrec)
        
        
        py_ayList.append(np.nan)
        py_tarList.append(np.nan)
        py_recList.append(np.nan)
        py_ydsList.append(np.nan)
        py_yacList.append(np.nan)
        py_tdList.append(np.nan)
        py_adotList.append(np.nan)
        py_racrList.append(np.nan)
        
        py_msAirList.append(np.nan)
        py_tgtShare.append(np.nan)
        py_woprList.append(np.nan)
        py_pprPointsList.append(np.nan)
        py_uayList.append(np.nan)
        py_catchRateList.append(np.nan)
        py_yptList.append(np.nan)
        py_yprList.append(np.nan)
        py_ayPerRecList.append(np.nan)
        py_ayPerTgtList.append(np.nan)
        py_uayPerTgtList.append(np.nan)
        py_uayPerRecList.append(np.nan)
        py_yacPerTgtList.append(np.nan)
        py_yacPerRecList.append(np.nan)
        
        
    else:
        
       
        name_list.append(name)
        name_list2.append(name)
        yearList.append(year)
        
        tgtList.append(tar)
        recList.append(rec)
        recYdsList.append(yds)
        ayList.append(AY)
        yacList.append(yac)
        tdList.append(td)
        adotList.append(adot)
        racrList.append(racr)
        msAirList.append(ms_air)
        tgtShareList.append(tgt_share)
        woprList.append(wopr)
        pprPointsList.append(ppr)
        uayList.append(UAY)
        catchRateList.append(catch_rate)
        yptList.append(ypt)
        yprList.append(ypr)
        ayPerRecList.append(ayrec)
        ayPerTgtList.append(aytar)
        uayPerTgtList.append(uaytar)
        uayPerRecList.append(uayrec)
        yacPerTgtList.append(yactgt)
        yacPerRecList.append(yacrec)
        
        
        py_ayList.append(AY)
        py_tarList.append(tar)
        py_recList.append(rec)
        py_ydsList.append(yds)
        py_yacList.append(yac)
        py_tdList.append(td)
        py_adotList.append(adot)
        py_racrList.append(racr)
        
        py_msAirList.append(ms_air)
        py_tgtShare.append(tgt_share)
        py_woprList.append(wopr)
        py_pprPointsList.append(ppr)
        py_uayList.append(UAY)
        py_catchRateList.append(catch_rate)
        py_yptList.append(ypt)
        py_yprList.append(ypr)
        py_ayPerRecList.append(ayrec)
        py_ayPerTgtList.append(aytar)
        py_uayPerTgtList.append(uaytar)
        py_uayPerRecList.append(uayrec)
        py_yacPerTgtList.append(yactgt)
        py_yacPerRecList.append(yacrec)

            

py_ayList.append(np.nan)
py_ayList.pop(0)
py_tarList.append(np.nan)
py_tarList.pop(0)
py_recList.append(np.nan)
py_recList.pop(0)
py_ydsList.append(np.nan)
py_ydsList.pop(0)
py_yacList.append(np.nan)
py_yacList.pop(0)
py_tdList.append(np.nan)
py_tdList.pop(0)
py_adotList.append(np.nan)
py_adotList.pop(0)
py_racrList.append(np.nan)
py_racrList.pop(0)

py_msAirList.append(np.nan)
py_msAirList.pop(0)
py_tgtShare.append(np.nan)
py_tgtShare.pop(0)
py_woprList.append(np.nan)
py_woprList.pop(0)
py_pprPointsList.append(np.nan)
py_pprPointsList.pop(0)
py_uayList.append(np.nan)
py_uayList.pop(0)
py_catchRateList.append(np.nan)
py_catchRateList.pop(0)
py_yptList.append(np.nan)
py_yptList.pop(0)
py_yprList.append(np.nan)
py_yprList.pop(0)
py_ayPerRecList.append(np.nan)
py_ayPerRecList.pop(0)
py_ayPerTgtList.append(np.nan)
py_ayPerTgtList.pop(0)
py_uayPerTgtList.append(np.nan)
py_uayPerTgtList.pop(0)
py_uayPerRecList.append(np.nan)
py_uayPerRecList.pop(0)
py_yacPerTgtList.append(np.nan)
py_yacPerTgtList.pop(0)
py_yacPerRecList.append(np.nan)
py_yacPerRecList.pop(0)

print("success")

success


In [10]:
TestList = [name_list, name_list2, yearList, tgtList, recList, recYdsList, ayList, yacList, tdList, adotList, racrList,
            msAirList, tgtShareList, woprList, pprPointsList, uayList, catchRateList, yptList, yprList, ayPerRecList, 
            ayPerTgtList, uayPerTgtList, uayPerRecList, yacPerTgtList, yacPerRecList, py_ayList, py_tarList, py_recList, 
            py_ydsList, py_yacList, py_tdList, py_adotList, py_racrList, 
            
            py_msAirList, py_tgtShare, py_woprList, py_pprPointsList, py_uayList, py_catchRateList, py_yptList, py_yprList, 
            py_ayPerRecList, py_ayPerTgtList, py_uayPerTgtList, py_uayPerRecList, py_yacPerTgtList, py_yacPerRecList]


lengthList = []

for list in TestList:
    lengthList.append(len(list))
    
lengthList

[1103,
 1103,
 1103,
 1103,
 1103,
 1103,
 1103,
 1103,
 1103,
 1103,
 1103,
 1103,
 1103,
 1103,
 1103,
 1103,
 1103,
 1103,
 1103,
 1103,
 1103,
 1103,
 1103,
 1103,
 1103,
 1103,
 1103,
 1103,
 1103,
 1103,
 1103,
 1103,
 1103,
 1103,
 1103,
 1103,
 1103,
 1103,
 1103,
 1103,
 1103,
 1103,
 1103,
 1103,
 1103,
 1103,
 1103]

In [11]:
combinedDict = {"full_name": name_list2, 
                "Year": yearList, 
                "PPR_Points": pprPointsList, 
                "Targets": tgtList,
                "Rec": recList, 
                "Air_Yards": ayList, 
                "YAC": yacList, 
                "TDs": tdList, 
                "ADOT": adotList, 
                "RACR": racrList, 
                "MS_Air": msAirList, 
                "Tgt_Share": tgtShareList, 
                "WOPR": woprList, 
                "UAY": uayList, 
                "Catch_Rate": catchRateList,
                "Yds/Tgt": yptList, 
                "Yds/Rec": yprList, 
                "AY/Tgt": ayPerTgtList, 
                "AY/Rec": ayPerRecList, 
                "UAY/tgt": uayPerTgtList, 
                "UAY/Rec": uayPerRecList, 
                "YAC/Tgt": yacPerTgtList, 
                "YAC/Rec": yacPerRecList, 
                "PY_AY": py_ayList, 
                "PY_Tgt":py_tarList, 
                "PY_Rec": py_recList, 
                "PY_Yds": py_ydsList, 
                "PY_YAC": py_yacList, 
                "PY_TD": py_tdList, 
                "PY_aDOT": py_adotList,
                "PY_RACR": py_racrList, 
                "PY_MS_AY": py_msAirList, 
                "PY_TGT_Share": py_tgtShare, 
                "PY_WOPR": py_woprList, 
                "PY_PPR_Points": py_pprPointsList, 
                "PY_UAY": py_uayList, 
                "PY_Catch_Rate": py_catchRateList, 
                "PY_Yds/TGT": py_yptList, 
                "PY_YPR": py_yprList, 
                "PY_AY/Tgt": py_ayPerTgtList, 
                "PY_AY/Rec": py_ayPerRecList,
                "PY_UAY/Tgt": py_uayPerTgtList, 
                "PY_UAY/Rec": py_uayPerRecList, 
                "PY_YAC/Tgt": py_yacPerTgtList, 
                "PY_YAC/Rec": py_yacPerRecList}


combinedDF = pd.DataFrame(combinedDict)

#combinedDF.replace([np.inf, -np.inf], np.nan)

combinedDF.sort_values(by=['full_name', 'Year'], ascending=False)
combinedDF

full_name  Year  PPR_Points  Targets  Rec  Air_Yards  YAC  TDs  \
0       Zay Jones  2019        35.0       27   20        234   58    0   
1       Zay Jones  2019        13.9       18    7        250   19    0   
2       Zay Jones  2018       152.1      100   54       1256  133    6   
3       Zay Jones  2017        70.6       74   27        994   63    2   
4     Zach Pascal  2019       133.3       72   41        800  235    5   
...           ...   ...         ...      ...  ...        ...  ...  ...   
1098   A.J. Green  2014       209.3      116   69       1511  309    6   
1099   A.J. Green  2013       306.6      178   98       2232  390   11   
1100   A.J. Green  2012       301.8      164   97       2239  361   11   
1101   A.J. Green  2011       218.0      115   65       1516  278    7   
1102   A.J. Brown  2019       217.1       84   52       1107  465    8   

           ADOT  RACR  ...  PY_UAY  PY_Catch_Rate  PY_Yds/TGT     PY_YPR  \
0      8.666667  0.63  ...   200.0      38.888889    3.833333   9.857143   
1     13.888889  0.28  ...   768.0      54.000000    6.210000  11.500000   
2     12.560000  0.49  ...   741.0      36.486486    4.270270  11.703704   
3     13.432432  0.32  ...     NaN            NaN         NaN        NaN   
4     11.111111  0.76  ...   269.0      59.090909    6.181818  10.461538   
...         ...   ...  ...     ...            ...         ...        ...   
1098  13.025862  0.69  ...  1196.0      55.056180    8.011236  14.551020   
1099  12.539326  0.64  ...  1250.0      59.146341    8.231707  13.917526   
1100  13.652439  0.60  ...   737.0      56.521739    9.191304  16.261538   
1101  13.182609  0.70  ...     NaN            NaN         NaN        NaN   
1102  13.178571  0.95  ...     NaN            NaN         NaN        NaN   

      PY_AY/Tgt  PY_AY/Rec  PY_UAY/Tgt  PY_UAY/Rec  PY_YAC/Tgt  PY_YAC/Rec  
0     13.888889  35.714286   11.111111   28.571429    1.055556    2.714286  
1     12.560000  23.259259    7.680000   14.222222    1.330000    2.462963  
2     13.432432  36.814815   10.013514   27.444444    0.851351    2.333333  
3           NaN        NaN         NaN         NaN         NaN         NaN  
4     10.681818  18.076923    6.113636   10.346154    1.613636    2.730769  
...         ...        ...         ...         ...         ...         ...  
1098  12.539326  22.775510    6.719101   12.204082    2.191011    3.979592  
1099  13.652439  23.082474    7.621951   12.886598    2.201220    3.721649  
1100  13.182609  23.323077    6.408696   11.338462    2.417391    4.276923  
1101        NaN        NaN         NaN         NaN         NaN         NaN  
1102        NaN        NaN         NaN         NaN         NaN         NaN  

[1103 rows x 45 columns]

In [12]:
cleaned_df = combinedDF

print(cleaned_df.isnull().sum())
print(cleaned_df.dtypes)

full_name          0
Year               0
PPR_Points         0
Targets            0
Rec                0
Air_Yards          0
YAC                0
TDs                0
ADOT              35
RACR              36
MS_Air             0
Tgt_Share          0
WOPR               0
UAY                0
Catch_Rate        37
Yds/Tgt           37
Yds/Rec           59
AY/Tgt            35
AY/Rec            36
UAY/tgt           35
UAY/Rec           36
YAC/Tgt           37
YAC/Rec           59
PY_AY            302
PY_Tgt           302
PY_Rec           302
PY_Yds           302
PY_YAC           302
PY_TD            302
PY_aDOT          328
PY_RACR          329
PY_MS_AY         302
PY_TGT_Share     302
PY_WOPR          302
PY_PPR_Points    302
PY_UAY           302
PY_Catch_Rate    329
PY_Yds/TGT       329
PY_YPR           345
PY_AY/Tgt        328
PY_AY/Rec        329
PY_UAY/Tgt       328
PY_UAY/Rec       329
PY_YAC/Tgt       329
PY_YAC/Rec       345
dtype: int64
full_name         object
Year             

In [13]:
#cleaned_df['PY_Tgt'] = cleaned_df['PY_Tgt'].astype(float)

#print(cleaned_df['PY_Tgt'].dtypes)
cleaned_df['PY_Tgt']

0        18.0
1       100.0
2        74.0
3         NaN
4        44.0
        ...  
1098    178.0
1099    164.0
1100    115.0
1101      NaN
1102      NaN
Name: PY_Tgt, Length: 1103, dtype: float64

In [14]:
# cleaned_df = cleaned_df[['Year', 'targets', 'rec', 'Catch %', 'rec_yards', 'Yds/tgt', 'Yds/rec', 'air_yards', 'adot', 
#                          'AY/rec', 'yac', 'YAC/tgt', 'YAC/rec', 'UAY', 'UAY/tgt', 'UAY/rec', 'racr',
#                          'ms_air', 'tgt_share', 'wopr', 'ppr']]

# cleaned_df = cleaned_df.rename(columns={'targets':'Targets', 'rec':'Rec', 'Catch %': 'Catch_Rate',
#                                         'rec_yards':'Rec_Yds', 'air_yards':'Air_Yds',
#                                         'yac':'YAC', 'adot':'aDOT', 'racr':'RACR', 'ms_air':'MS_AY',
#                                         'tgt_share':'Tgt_Share','wopr':'WOPR', 'ppr': 'PPR_Points'})

cleaned_df.replace([np.inf, -np.inf], np.nan)

cleaned_df = cleaned_df.dropna()

#cleaned_df = cleaned_df.drop(cleaned_df[cleaned_df[cleaned_df['PY_Tgt']] < 0].index, inplace = True)

cleaned_df = cleaned_df.round(2)

cleaned_df.drop(cleaned_df.tail(1).index,inplace=True)

cleaned_df

full_name  Year  PPR_Points  Targets  Rec  Air_Yards  YAC  TDs  \
0        Zay Jones  2019        35.0       27   20        234   58    0   
1        Zay Jones  2019        13.9       18    7        250   19    0   
2        Zay Jones  2018       152.1      100   54       1256  133    6   
4      Zach Pascal  2019       133.3       72   41        800  235    5   
6     Willie Snead  2019        95.1       46   31        409  126    5   
...            ...   ...         ...      ...  ...        ...  ...  ...   
1095    A.J. Green  2017       230.8      143   75       1910  297    8   
1096    A.J. Green  2016       186.4      100   66       1236  258    4   
1097    A.J. Green  2015       275.7      132   86       1687  336   10   
1098    A.J. Green  2014       209.3      116   69       1511  309    6   
1099    A.J. Green  2013       306.6      178   98       2232  390   11   

       ADOT  RACR  ...  PY_UAY  PY_Catch_Rate  PY_Yds/TGT  PY_YPR  PY_AY/Tgt  \
0      8.67  0.63  ...   200.0          38.89        3.83    9.86      13.89   
1     13.89  0.28  ...   768.0          54.00        6.21   11.50      12.56   
2     12.56  0.49  ...   741.0          36.49        4.27   11.70      13.43   
4     11.11  0.76  ...   269.0          59.09        6.18   10.46      10.68   
6      8.89  0.83  ...   402.0          65.26        6.83   10.47       8.16   
...     ...   ...  ...     ...            ...         ...     ...        ...   
1095  13.36  0.56  ...   530.0          66.00        9.64   14.61      12.36   
1096  12.36  0.78  ...   726.0          65.15        9.83   15.08      12.78   
1097  12.78  0.77  ...   779.0          59.48        8.97   15.09      13.03   
1098  13.03  0.69  ...  1196.0          55.06        8.01   14.55      12.54   
1099  12.54  0.64  ...  1250.0          59.15        8.23   13.92      13.65   

      PY_AY/Rec  PY_UAY/Tgt  PY_UAY/Rec  PY_YAC/Tgt  PY_YAC/Rec  
0         35.71       11.11       28.57        1.06        2.71  
1         23.26        7.68       14.22        1.33        2.46  
2         36.81       10.01       27.44        0.85        2.33  
4         18.08        6.11       10.35        1.61        2.73  
6         12.50        4.23        6.48        2.91        4.45  
...         ...         ...         ...         ...         ...  
1095      18.73        5.30        8.03        2.58        3.91  
1096      19.62        5.50        8.44        2.55        3.91  
1097      21.90        6.72       11.29        2.66        4.48  
1098      22.78        6.72       12.20        2.19        3.98  
1099      23.08        7.62       12.89        2.20        3.72  

[742 rows x 45 columns]

In [15]:
###Clean DF for ML###

#cleaned_df = cleaned_df.drop(columns=["full_name"])

cleaned_df

full_name  Year  PPR_Points  Targets  Rec  Air_Yards  YAC  TDs  \
0        Zay Jones  2019        35.0       27   20        234   58    0   
1        Zay Jones  2019        13.9       18    7        250   19    0   
2        Zay Jones  2018       152.1      100   54       1256  133    6   
4      Zach Pascal  2019       133.3       72   41        800  235    5   
6     Willie Snead  2019        95.1       46   31        409  126    5   
...            ...   ...         ...      ...  ...        ...  ...  ...   
1095    A.J. Green  2017       230.8      143   75       1910  297    8   
1096    A.J. Green  2016       186.4      100   66       1236  258    4   
1097    A.J. Green  2015       275.7      132   86       1687  336   10   
1098    A.J. Green  2014       209.3      116   69       1511  309    6   
1099    A.J. Green  2013       306.6      178   98       2232  390   11   

       ADOT  RACR  ...  PY_UAY  PY_Catch_Rate  PY_Yds/TGT  PY_YPR  PY_AY/Tgt  \
0      8.67  0.63  ...   200.0          38.89        3.83    9.86      13.89   
1     13.89  0.28  ...   768.0          54.00        6.21   11.50      12.56   
2     12.56  0.49  ...   741.0          36.49        4.27   11.70      13.43   
4     11.11  0.76  ...   269.0          59.09        6.18   10.46      10.68   
6      8.89  0.83  ...   402.0          65.26        6.83   10.47       8.16   
...     ...   ...  ...     ...            ...         ...     ...        ...   
1095  13.36  0.56  ...   530.0          66.00        9.64   14.61      12.36   
1096  12.36  0.78  ...   726.0          65.15        9.83   15.08      12.78   
1097  12.78  0.77  ...   779.0          59.48        8.97   15.09      13.03   
1098  13.03  0.69  ...  1196.0          55.06        8.01   14.55      12.54   
1099  12.54  0.64  ...  1250.0          59.15        8.23   13.92      13.65   

      PY_AY/Rec  PY_UAY/Tgt  PY_UAY/Rec  PY_YAC/Tgt  PY_YAC/Rec  
0         35.71       11.11       28.57        1.06        2.71  
1         23.26        7.68       14.22        1.33        2.46  
2         36.81       10.01       27.44        0.85        2.33  
4         18.08        6.11       10.35        1.61        2.73  
6         12.50        4.23        6.48        2.91        4.45  
...         ...         ...         ...         ...         ...  
1095      18.73        5.30        8.03        2.58        3.91  
1096      19.62        5.50        8.44        2.55        3.91  
1097      21.90        6.72       11.29        2.66        4.48  
1098      22.78        6.72       12.20        2.19        3.98  
1099      23.08        7.62       12.89        2.20        3.72  

[742 rows x 45 columns]

In [16]:
#Confirm no nulls

cleaned_df.count()

full_name        742
Year             742
PPR_Points       742
Targets          742
Rec              742
Air_Yards        742
YAC              742
TDs              742
ADOT             742
RACR             742
MS_Air           742
Tgt_Share        742
WOPR             742
UAY              742
Catch_Rate       742
Yds/Tgt          742
Yds/Rec          742
AY/Tgt           742
AY/Rec           742
UAY/tgt          742
UAY/Rec          742
YAC/Tgt          742
YAC/Rec          742
PY_AY            742
PY_Tgt           742
PY_Rec           742
PY_Yds           742
PY_YAC           742
PY_TD            742
PY_aDOT          742
PY_RACR          742
PY_MS_AY         742
PY_TGT_Share     742
PY_WOPR          742
PY_PPR_Points    742
PY_UAY           742
PY_Catch_Rate    742
PY_Yds/TGT       742
PY_YPR           742
PY_AY/Tgt        742
PY_AY/Rec        742
PY_UAY/Tgt       742
PY_UAY/Rec       742
PY_YAC/Tgt       742
PY_YAC/Rec       742
dtype: int64

In [17]:
cleaned_df.to_csv('test.csv', index=False, header=True)

#ValueError: Input contains infinity or a value too large for dtype('float64').

cleaned_df.dtypes

full_name         object
Year               int64
PPR_Points       float64
Targets            int64
Rec                int64
Air_Yards          int64
YAC                int64
TDs                int64
ADOT             float64
RACR             float64
MS_Air           float64
Tgt_Share        float64
WOPR             float64
UAY                int64
Catch_Rate       float64
Yds/Tgt          float64
Yds/Rec          float64
AY/Tgt           float64
AY/Rec           float64
UAY/tgt          float64
UAY/Rec          float64
YAC/Tgt          float64
YAC/Rec          float64
PY_AY            float64
PY_Tgt           float64
PY_Rec           float64
PY_Yds           float64
PY_YAC           float64
PY_TD            float64
PY_aDOT          float64
PY_RACR          float64
PY_MS_AY         float64
PY_TGT_Share     float64
PY_WOPR          float64
PY_PPR_Points    float64
PY_UAY           float64
PY_Catch_Rate    float64
PY_Yds/TGT       float64
PY_YPR           float64
PY_AY/Tgt        float64


In [18]:
for col in cleaned_df.columns: 
    print(col)

full_name
Year
PPR_Points
Targets
Rec
Air_Yards
YAC
TDs
ADOT
RACR
MS_Air
Tgt_Share
WOPR
UAY
Catch_Rate
Yds/Tgt
Yds/Rec
AY/Tgt
AY/Rec
UAY/tgt
UAY/Rec
YAC/Tgt
YAC/Rec
PY_AY
PY_Tgt
PY_Rec
PY_Yds
PY_YAC
PY_TD
PY_aDOT
PY_RACR
PY_MS_AY
PY_TGT_Share
PY_WOPR
PY_PPR_Points
PY_UAY
PY_Catch_Rate
PY_Yds/TGT
PY_YPR
PY_AY/Tgt
PY_AY/Rec
PY_UAY/Tgt
PY_UAY/Rec
PY_YAC/Tgt
PY_YAC/Rec


In [19]:
print("Glossary:")
print("-----------------------------------------------------------")
print("-----------------------------------------------------------")
print("Targets (tgt): # of times the ball was thrown to the WR")
print("-----------------------------------------------------------")
print("Rec (Receptions): # of times the receiver caught the ball")
print("-----------------------------------------------------------")
print("Catch %: Rec/Targets")
print("-----------------------------------------------------------")
print("Rec_Yds (Receiving Yards): Total yards gained on all Recs")
print("-----------------------------------------------------------")
print("Air_Yds: Total distance from the line of scrimmage to where the WR is located at the time the WR is targeted")
print("-----------------------------------------------------------")
print("YAC (Yards After Catch): Yards gained after the receiver catches the ball")
print("-----------------------------------------------------------")
print("UAY (Unrealized Air Yards): total AY on tgts that did not result in a rec (Air_Yds - (Rec_Yds-YAC)")
print("-----------------------------------------------------------")
print("aDOT (Average Depth of Target): Average distance from the line of scrimmage to the point at at which the WR is targeted")
print("-----------------------------------------------------------")
print("RACR (receiver air conversion ratio): Measures yards created per AY (Rec_Yds/Air_Yds)")
print("-----------------------------------------------------------")
print("MS_AY (Market Share of Air Yards): % of total team air yards on tgts to the WR")
print("-----------------------------------------------------------")
print("Tgt_Share (Target Share): % of team targets intended for the WR")
print("-----------------------------------------------------------")
print("WOPR: weighted opportunity rating")
print("-----------------------------------------------------------")
print("PPR_Points: fantasy points scored in PPR (Points Per Reception) scoring format. PPR is the standard scoring format for ESPN")
print("-----------------------------------------------------------")
print("PY: Prior Year")

Glossary:
-----------------------------------------------------------
-----------------------------------------------------------
Targets (tgt): # of times the ball was thrown to the WR
-----------------------------------------------------------
Rec (Receptions): # of times the receiver caught the ball
-----------------------------------------------------------
Catch %: Rec/Targets
-----------------------------------------------------------
Rec_Yds (Receiving Yards): Total yards gained on all Recs
-----------------------------------------------------------
Air_Yds: Total distance from the line of scrimmage to where the WR is located at the time the WR is targeted
-----------------------------------------------------------
YAC (Yards After Catch): Yards gained after the receiver catches the ball
-----------------------------------------------------------
UAY (Unrealized Air Yards): total AY on tgts that did not result in a rec (Air_Yds - (Rec_Yds-YAC)
-----------------------------------

In [20]:
#X1 = cleaned_df.Targets.values.reshape(-1, 1)
#y = cleaned_df.PPR_Points.values.reshape(-1, 1) 

#print("Shape: ", X1.shape, y.shape)
#X1

In [21]:
#plt.scatter(X1, y)

In [22]:
#X2 = cleaned_df.Air_Yds.values.reshape(-1, 1)
#y = cleaned_df.PPR_Points.values.reshape(-1, 1) 

#print("Shape: ", X2.shape, y.shape)
#X2

In [23]:
#plt.scatter(X2, y)

In [24]:
#X3 = cleaned_df.MS_AY.values.reshape(-1, 1)
#y = cleaned_df.PPR_Points.values.reshape(-1, 1) 

#print("Shape: ", X3.shape, y.shape)
#X3

#print(len(y))
#y

#ms_ayDF = cleaned_df["MS_AY"]

In [25]:
#plt.scatter(X3, y)

In [26]:
# Create the model and fit the model to the data
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn import datasets
model = LinearRegression()
model

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [27]:
pd.set_option('display.max_rows', 1000)

cleaned_df['PY_aDOT']

#pandas.set_option('display.max_rows', 10)

0       13.89
1       12.56
2       13.43
4       10.68
6        8.16
7        6.69
8        7.53
9       10.12
11      13.84
12      15.88
13      15.52
23      12.52
24      14.32
25      11.61
26      26.83
28      14.38
29      11.69
30       7.98
32      13.58
33      12.99
34      10.58
35      13.38
37      10.06
38       6.81
39       8.32
41       7.30
43      18.00
47       9.00
49       7.10
51      11.48
53      11.81
56      22.87
57      15.57
58      13.84
59      11.50
60      18.59
61      18.69
62      14.43
64      13.47
65      14.02
66      14.76
67      15.49
68      18.20
69      17.81
71       6.82
73      10.64
79      11.74
80      13.63
81      15.45
82      13.47
84      16.57
85      12.06
86      12.31
87      16.18
88      15.35
89      14.50
90      -2.00
91      11.18
92      15.03
93      15.88
96      14.46
98      11.53
99      10.96
100     10.43
101      7.50
102     13.59
104     17.57
105      1.74
106      6.89
107      7.31
108      4.07
109   

In [28]:
cleaned_df.iloc[:79]

full_name  Year  PPR_Points  Targets  Rec  Air_Yards  YAC  TDs  \
0            Zay Jones  2019        35.0       27   20        234   58    0   
1            Zay Jones  2019        13.9       18    7        250   19    0   
2            Zay Jones  2018       152.1      100   54       1256  133    6   
4          Zach Pascal  2019       133.3       72   41        800  235    5   
6         Willie Snead  2019        95.1       46   31        409  126    5   
7         Willie Snead  2018       134.2       95   62        775  276    1   
8         Willie Snead  2017        17.2       16    8        107   32    0   
9         Willie Snead  2016       185.5      104   72        783  387    4   
11         Will Fuller  2019       134.0       71   49       1011  219    3   
12         Will Fuller  2018       106.3       45   32        623  166    4   
13         Will Fuller  2017       113.2       50   28        794   99    7   
23     Tyrell Williams  2019       143.1       64   42        870  190    6   
24     Tyrell Williams  2018       134.5       63   40        789  183    5   
25     Tyrell Williams  2017       139.8       69   43        988  320    4   
26     Tyrell Williams  2016       216.9      119   69       1382  433    7   
28         Tyreek Hill  2019       188.3       89   58       1149  275    7   
29         Tyreek Hill  2018       294.0      132   82       1898  520   10   
30         Tyreek Hill  2017       241.2      105   75       1227  451    7   
32       Tyler Lockett  2019       235.2      110   82       1364  308    8   
33       Tyler Lockett  2018       217.1       67   55        910  227   10   
34       Tyler Lockett  2017       118.3       71   45        922  188    2   
35       Tyler Lockett  2016       124.1       66   41        698  216    1   
37          Tyler Boyd  2019       226.9      148   90       1186  423    5   
38          Tyler Boyd  2018       221.1      108   76       1087  426    7   
39          Tyler Boyd  2017        56.5       32   22        218   91    2   
41          Trey Quinn  2019        51.8       47   26        324   61    1   
43        Trevor Davis  2019        28.6       10    7         82   55    0   
47        Trevor Davis  2017        13.3        7    5         69   34    0   
49        Trent Taylor  2018        53.5       40   26        254  102    1   
51     Trent Sherfield  2019        12.0       13    4        195   10    0   
53      Tre'Quan Smith  2019        71.4       25   18        226  111    5   
56     Travis Benjamin  2019         9.0       16    6        214   19    0   
57     Travis Benjamin  2018        39.6       23   11        526   60    1   
58     Travis Benjamin  2017       124.3       65   34       1012  210    4   
59     Travis Benjamin  2016       138.4       75   47       1038  246    4   
60     Travis Benjamin  2015       195.8      125   68       1438  354    5   
61     Travis Benjamin  2014        68.5       46   18        855   43    3   
62     Travis Benjamin  2013        20.0       13    5        243   60    0   
64        Torrey Smith  2017        90.7       68   36        877  135    2   
65        Torrey Smith  2016        64.7       49   20        660   61    3   
66        Torrey Smith  2015       123.3       62   33        869  213    4   
67        Torrey Smith  2014       191.7       92   49       1358  169   11   
68        Torrey Smith  2013       201.8      137   65       2122  359    4   
69        Torrey Smith  2012       183.4      110   49       2002  211    8   
71      Tommylee Lewis  2017        29.0       15   10        152   52    1   
73         Tim Patrick  2019        37.8       31   16        315   38    0   
79   Terrance Williams  2017       111.3       78   53        663  243    0   
80   Terrance Williams  2016       127.4       61   44        716  171    4   
81   Terrance Williams  2015       154.0       93   52       1268  275    3   
82   Terrance Williams  2014       149.3       66   38       10

In [29]:
#cleaned_df.PPR_Points.astype(str)

testDF = cleaned_df[['PY_Tgt', 
                     'PY_Rec', 
                     'PY_Yds', 
                     'PY_AY', 
                     'PY_aDOT', 
                     'PY_YAC', 
                     'PY_TD', 
                     'PY_RACR', 
                     'PY_MS_AY', 
                     'PY_TGT_Share',
                     'PY_WOPR',
                     'PY_UAY',
                     'PY_Catch_Rate',
                     'PY_Yds/TGT',
                     'PY_YPR',
                     'PY_AY/Tgt',
                     'PY_AY/Rec',
                     'PY_UAY/Tgt',
                     'PY_UAY/Rec',
                     'PY_YAC/Tgt',
                     'PY_YAC/Rec',
                     'PY_PPR_Points',                   
                     'PPR_Points']]

#'PY_Tgt', 'PY_Rec', 'PY_Yds', 'PY_YAC', 'PY_TD', *'PY_aDOT', 'PY_RACR', 

testDF['PPR_Points'] = testDF['PPR_Points'].astype(str)
testDF

C:\Users\Joseph\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


PY_Tgt  PY_Rec  PY_Yds   PY_AY  PY_aDOT  PY_YAC  PY_TD  PY_RACR  \
0       18.0     7.0    69.0   250.0    13.89    19.0    0.0     0.28   
1      100.0    54.0   621.0  1256.0    12.56   133.0    6.0     0.49   
2       74.0    27.0   316.0   994.0    13.43    63.0    2.0     0.32   
4       44.0    26.0   272.0   470.0    10.68    71.0    2.0     0.58   
6       95.0    62.0   649.0   775.0     8.16   276.0    1.0     0.84   
7       16.0     8.0    92.0   107.0     6.69    32.0    0.0     0.86   
8      104.0    72.0   895.0   783.0     7.53   387.0    4.0     1.14   
9      102.0    70.0   990.0  1032.0    10.12   344.0    3.0     0.96   
11      45.0    32.0   503.0   623.0    13.84   166.0    4.0     0.81   
12      50.0    28.0   423.0   794.0    15.88    99.0    7.0     0.53   
13      92.0    47.0   635.0  1428.0    15.52   238.0    2.0     0.44   
23      63.0    40.0   631.0   789.0    12.52   183.0    5.0     0.80   
24      69.0    43.0   728.0   988.0    14.32   320.0    4.0     0.74   
25     119.0    69.0  1059.0  1382.0    11.61   433.0    7.0     0.77   
26       6.0     2.0    90.0   161.0    26.83    51.0    1.0     0.56   
28     132.0    82.0  1336.0  1898.0    14.38   520.0   10.0     0.70   
29     105.0    75.0  1183.0  1227.0    11.69   451.0    7.0     0.96   
30      83.0    61.0   593.0   662.0     7.98   268.0    6.0     0.90   
32      67.0    55.0   970.0   910.0    13.58   227.0   10.0     1.07   
33      71.0    45.0   555.0   922.0    12.99   188.0    2.0     0.60   
34      66.0    41.0   597.0   698.0    10.58   216.0    1.0     0.86   
35      69.0    51.0   664.0   923.0    13.38   249.0    6.0     0.72   
37     108.0    76.0  1028.0  1087.0    10.06   426.0    7.0     0.95   
38      32.0    22.0   225.0   218.0     6.81    91.0    2.0     1.03   
39      81.0    54.0   603.0   674.0     8.32   201.0    1.0     0.89   
41      10.0     9.0    75.0    73.0     7.30    17.0    1.0     1.03   
43       1.0     1.0    28.0    18.0    18.00    10.0    0.0     1.56   
47       7.0     3.0    24.0    63.0     9.00     8.0    1.0     0.38   
49      60.0    43.0   430.0   426.0     7.10   190.0    2.0     1.01   
51      25.0    16.0   171.0   287.0    11.48    27.0    1.0     0.60   
53      42.0    26.0   350.0   496.0    11.81    78.0    3.0     0.71   
56      23.0    11.0   185.0   526.0    22.87    60.0    1.0     0.35   
57      65.0    34.0   567.0  1012.0    15.57   210.0    4.0     0.56   
58      75.0    47.0   677.0  1038.0    13.84   246.0    4.0     0.65   
59     125.0    68.0   966.0  1438.0    11.50   354.0    5.0     0.67   
60      46.0    18.0   314.0   855.0    18.59    43.0    3.0     0.37   
61      13.0     5.0   105.0   243.0    18.69    60.0    0.0     0.43   
62      37.0    18.0   298.0   534.0    14.43    69.0    2.0     0.56   
64      49.0    20.0   267.0   660.0    13.47    61.0    3.0     0.40   
65      62.0    33.0   663.0   869.0    14.02   213.0    4.0     0.76   
66      92.0    49.0   767.0  1358.0    14.76   169.0   11.0     0.56   
67     137.0    65.0  1128.0  2122.0    15.49   359.0    4.0     0.53   
68     110.0    49.0   855.0  2002.0    18.20   211.0    8.0     0.43   
69      95.0    50.0   841.0  1692.0    17.81   238.0    7.0     0.50   
71      11.0     7.0    75.0    75.0     6.82    20.0    0.0     1.00   
73      42.0    24.0   319.0   447.0    10.64   112.0    2.0     0.71   
79      61.0    44.0   594.0   716.0    11.74   171.0    4.0     0.83   
80      93.0    52.0   840.0  1268.0    13.63   275.0    3.0     0.66   
81      66.0    38.0   633.0  1020.0    15.45   132.0    8.0     0.62   
82      74.0    44.0   736.0   997.0    13.47   204.0    5.0     0.74   
84      28.0    16.0   207.0   464.0    16.57    43.0    2.0     0.45   
85      70.0    53.0   787.0   844.0    12.06   302.0    4.0     0.93   
86      95.0    54.0   752.0  1169.0    12.31   182.0    4.0     0.64   
87      97.0    44.0   739.0  1569.0    16.18   241.0   10.0    

In [30]:
# Create X and y for initial ML model

X =  testDF.drop(columns=['PPR_Points'])

y = testDF[["PPR_Points"]]



Model #1: Random Forest on All Features

In [31]:
# Create RandomForestRegressor Model and split data into training and testing data

from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=250)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

type(X)
print(y.dtypes)
print(X.dtypes)

PPR_Points    object
dtype: object
PY_Tgt           float64
PY_Rec           float64
PY_Yds           float64
PY_AY            float64
PY_aDOT          float64
PY_YAC           float64
PY_TD            float64
PY_RACR          float64
PY_MS_AY         float64
PY_TGT_Share     float64
PY_WOPR          float64
PY_UAY           float64
PY_Catch_Rate    float64
PY_Yds/TGT       float64
PY_YPR           float64
PY_AY/Tgt        float64
PY_AY/Rec        float64
PY_UAY/Tgt       float64
PY_UAY/Rec       float64
PY_YAC/Tgt       float64
PY_YAC/Rec       float64
PY_PPR_Points    float64
dtype: object


In [32]:
# Fit RandomFoirest model to the training data


model.fit(X_train, y_train)
#model.score(X_test, y_test)

C:\Users\Joseph\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=250,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [33]:
# Calculate feature importance
importances = model.feature_importances_
importances

array([0.03378753, 0.0341926 , 0.14103384, 0.02896974, 0.01179648,
       0.06342526, 0.01711451, 0.02098905, 0.02638204, 0.04987278,
       0.21638114, 0.02325821, 0.03171928, 0.04270992, 0.02922537,
       0.01169618, 0.01547768, 0.01880472, 0.0149424 , 0.03208339,
       0.03003873, 0.10609913])

In [34]:
# Sort the features by their importance
sorted(zip(model.feature_importances_, X.columns), reverse=True)

[(0.21638113544609983, 'PY_WOPR'),
 (0.1410338423754267, 'PY_Yds'),
 (0.10609913402239057, 'PY_PPR_Points'),
 (0.06342525582852557, 'PY_YAC'),
 (0.04987278409409645, 'PY_TGT_Share'),
 (0.04270992391967145, 'PY_Yds/TGT'),
 (0.03419259880030622, 'PY_Rec'),
 (0.03378753269408868, 'PY_Tgt'),
 (0.03208338825694952, 'PY_YAC/Tgt'),
 (0.03171928202993117, 'PY_Catch_Rate'),
 (0.03003872755114853, 'PY_YAC/Rec'),
 (0.02922536824273224, 'PY_YPR'),
 (0.02896973668332499, 'PY_AY'),
 (0.026382037242444492, 'PY_MS_AY'),
 (0.0232582137394998, 'PY_UAY'),
 (0.020989048841427174, 'PY_RACR'),
 (0.01880472327858991, 'PY_UAY/Tgt'),
 (0.01711451340802262, 'PY_TD'),
 (0.015477682902806602, 'PY_AY/Rec'),
 (0.014942404483148097, 'PY_UAY/Rec'),
 (0.011796481521717805, 'PY_aDOT'),
 (0.011696184637651512, 'PY_AY/Tgt')]

In [35]:
# Confirm shape of: X_train, X_test, y_train, y_test = train_test_split

print('Training Features Shape:', X_train.shape)
print('Training Labels Shape:', y_train.shape)
print('Testing Features Shape:', X_test.shape)
print('Testing Labels Shape:', y_test.shape)

Training Features Shape: (556, 22)
Training Labels Shape: (556, 1)
Testing Features Shape: (186, 22)
Testing Labels Shape: (186, 1)


In [36]:
# Make predictions using a fitted model


predictions = model.predict(X_test)
predictions

array([121.1836, 102.4688, 134.0516, 111.4548,  72.1696,  63.2932,
        73.4164, 116.9432, 249.3284, 192.3412,  91.3116,  52.2692,
       232.926 ,  58.5948,  98.7216, 147.0888, 223.7956,  73.676 ,
       186.2072, 120.7876, 142.634 , 221.6496, 190.978 ,  95.3484,
       221.7096, 202.0936, 117.6228, 187.8208, 189.3264,  66.6768,
       210.7796,  45.8284, 116.382 ,  29.9028, 176.1636,  87.4744,
       185.6888, 113.2464, 227.9292, 144.488 , 156.6092,  85.0848,
       238.9492,  40.884 , 156.0076, 221.0392, 158.2964, 154.4816,
       221.9468, 109.8456, 197.3564, 151.3472,  34.396 , 118.578 ,
       304.8132,  38.0504, 103.0268, 115.0396, 222.306 , 136.56  ,
       142.016 ,  59.0652, 148.984 , 262.6964, 151.0932,  67.5732,
       157.5176, 108.2348,  50.736 , 153.7564, 158.8612,  35.3496,
        57.9508,  78.3936,  58.5752,  68.4752, 149.0672,  34.084 ,
        62.386 , 128.5732,  60.1888, 303.8044, 150.466 ,  83.2808,
        57.6684, 194.9908,  92.4024,  55.3536, 211.352 , 148.8

In [37]:
# Score model using sklearn metrics package

from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, predictions))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, predictions))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))
print("Training Accuracy = ", model.score(X_train, y_train))
print("Test Accuracy = ", model.score(X_test, y_test))


Mean Absolute Error: 58.579853763440866
Mean Squared Error: 5642.706422577203
Root Mean Squared Error: 75.11795006905608
Training Accuracy =  0.9192231981921242
Test Accuracy =  0.3062515473412535


In [38]:
# Plot the difference between the model predicted values and actual y values, versus the model predicted values

Model #2: Random Forest w/ least important features removed

In [39]:
# Create 2nd RandomForest Model

model2 = RandomForestRegressor(n_estimators=250)

# Create X and y for 2nd ML model with unimportant features removed

X2 =  testDF[['PY_WOPR', 
             'PY_Yds', 
             'PY_PPR_Points', 
             'PY_YAC', 
             'PY_TGT_Share', 
             'PY_Yds/TGT', 
             'PY_Tgt', 
             'PY_Rec', 
             'PY_YAC/Tgt', 
             'PY_YAC/Rec',
             'PY_Catch_Rate']]

X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y, random_state=42)

# Fit RandomForest model to the culled training data

model2.fit(X_train2, y_train2)
#model2.score(X_test2, y_test2)

C:\Users\Joseph\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=250,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [40]:
# Calculate feature importance
importances2 = model2.feature_importances_
importances2

array([0.2389486 , 0.14887755, 0.15143363, 0.079199  , 0.05715808,
       0.07063941, 0.04825232, 0.04438082, 0.0508756 , 0.04998949,
       0.0602455 ])

In [41]:
# Sort the features by their importance
sorted(zip(model2.feature_importances_, X2.columns), reverse=True)

[(0.23894860324653128, 'PY_WOPR'),
 (0.15143362906123556, 'PY_PPR_Points'),
 (0.14887754683939036, 'PY_Yds'),
 (0.07919900057402025, 'PY_YAC'),
 (0.07063941202801298, 'PY_Yds/TGT'),
 (0.06024549925815326, 'PY_Catch_Rate'),
 (0.05715807643978786, 'PY_TGT_Share'),
 (0.050875600022677145, 'PY_YAC/Tgt'),
 (0.049989491936800294, 'PY_YAC/Rec'),
 (0.04825231690501595, 'PY_Tgt'),
 (0.044380823688375204, 'PY_Rec')]

In [42]:
# Make predictions using a 2nd model


predictions2 = model2.predict(X_test2)
predictions2

array([114.9344, 111.3104, 135.8376,  99.2032,  76.8364,  52.2004,
        74.9084,  93.9216, 257.9744, 180.0108, 104.1428,  55.5848,
       227.4924,  73.0804,  98.8228, 143.8544, 196.0148,  71.2352,
       185.5304, 109.47  , 147.01  , 212.3568, 186.0472,  98.7744,
       217.0416, 206.8272,  95.436 , 200.7288, 189.8212,  59.664 ,
       202.7008,  31.792 , 113.7084,  31.8312, 172.9616,  79.7944,
       181.7784, 108.3744, 221.4788, 140.74  , 146.4056,  86.9168,
       241.1592,  33.3536, 135.3984, 236.11  , 165.7188, 122.992 ,
       218.1188,  93.5948, 213.1068, 137.0636,  38.2076, 142.012 ,
       303.1992,  38.1864, 113.1468, 117.9028, 216.5768, 132.1112,
       130.3568,  57.0992, 134.6992, 262.4316, 142.1108,  87.854 ,
       149.798 , 103.9744,  53.1928, 137.9132, 169.6948,  33.8892,
        46.8724,  78.4212,  55.1532,  72.518 , 152.9996,  35.4916,
        76.448 , 124.528 ,  64.2004, 310.9652, 159.8756,  85.4108,
        69.4448, 190.8728,  93.104 ,  44.536 , 200.444 , 148.7

In [43]:
# Score model using sklearn metrics package


print('Mean Absolute Error:', metrics.mean_absolute_error(y_test2, predictions2))
print('Mean Squared Error:', metrics.mean_squared_error(y_test2, predictions2))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test2, predictions2)))
print("Training Accuracy = ", model2.score(X_train2, y_train2))
print("Test Accuracy = ", model2.score(X_test2, y_test2))

Mean Absolute Error: 58.687565591397856
Mean Squared Error: 5715.7000706443005
Root Mean Squared Error: 75.6022491110172
Training Accuracy =  0.9169537570654882
Test Accuracy =  0.2972772668084488


Model #3: Decision Tree Regressor w/ all features

In [44]:
# import the regressor 
from sklearn.tree import DecisionTreeRegressor

# create a regressor object 
model3 = DecisionTreeRegressor(random_state = 42)  
  
# fit the regressor with X and Y data 
model3.fit(X_train, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=42, splitter='best')

In [45]:
# predicting a new value 
   
predictions3 = model3.predict(X_test) 
predictions3

array([144.5, 154. ,  69.9,  90.3,  43.9, 131.3,  98.8,   2.7, 319.2,
       237.3, 105.4,   1.8, 251.9,  47.8,  14.7, 111.3, 245. ,  45.1,
       133.6, 139.2, 193.5, 200.2, 148.3,  95.8, 156.7, 232.3, 171.7,
       115.1,   4.9,  27.2, 232.9,  24.7,  36. ,  12.1, 275.7,   1.8,
       157.8,  95.1, 200.2,  59.1, 186. ,  69.9, 252.3,  15.6, 230. ,
       319.2, 225.1, 136.1, 228.4, 217.1, 279.4, 118.3,   4.7,  81.9,
       307.3,  99.2, 115.4, 148.7, 200.2, 100.8,  62.5,   1.3,  49.7,
       307.3,  78.7, 179.4, 148.3, 148.9,  62.4,  49.7, 278.9,  18. ,
       216.9, 119.8, 129.9, 115.2,  88.6,  18. ,   1.6, 161.3, 131.3,
       320.7, 284.5,  37.8, 107.6, 199.1,  97.6,  33.7, 196.7, 278.2,
        29.6, 153. , 114.4,  54.4,  26.4,  67. ,  12.6, 166.3, 136.1,
        85.6,   1.8,   5.7, 168.3, 338.9,  60.2, 244.9,  42.9,  53.5,
         5.5, 246.2,  93.5, 156.7,  61. , 320.7,  40.2, 180.8,  60.6,
        14.2,  38.8,  46.4,   1.8,  58. ,  45.1,  49.7, 235.2,  51.3,
       217.1,  49.7,

In [46]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, predictions3))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, predictions3))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, predictions3)))
print("Training Accuracy = ", model3.score(X_train, y_train))
print("Test Accuracy = ", model3.score(X_test, y_test))

Mean Absolute Error: 75.552688172043
Mean Squared Error: 9619.690430107526
Root Mean Squared Error: 98.08002054499951
Training Accuracy =  1.0
Test Accuracy =  -0.18270291791916105


Model #4: Decision Tree Regressor w/ least important features removed

In [47]:
# create a regressor object 
model4 = DecisionTreeRegressor(random_state = 42)  
  
# fit the regressor with X and Y data 
model4.fit(X_train2, y_train2)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=42, splitter='best')

In [48]:
# predicting a new value 
   
predictions4 = model4.predict(X_test2) 
predictions4

array([120.2, 200.8, 200.8,  37.8,   3.1, 131.3,   5. ,   2.7, 319.2,
       237.3, 105.4,  51.8, 251.9,  98.8,  84.8, 111.3, 232.9,  85.9,
       153.4,  39.6, 189. , 246.9,  67.8, 130.9, 156.7, 232.3, 200.8,
        62.5, 239.1,  27.2, 245. ,  15.6,  99.4,  40.2, 279.4,   1.5,
       157.8,  90.3, 232.9, 102.6, 186. , 200.8, 251.9,  19.3, 178.5,
       303. , 225.1,  60.9, 232.9,  95.1, 279.4,  92. ,  69.4, 148.8,
       307.3,   5.1, 142.2,  94.7, 252.3, 171.1, 129.4, 100.9, 115.3,
       307.3, 171.1,  61.8, 148.3,  14.2,  62.4,  49.7,  78.8,  18. ,
       216.9,  93.5,   1.8,   1.5, 139.2,  18. ,  98.8,  88.5, 131.3,
       271.4, 278.9, 141.9,  68.5, 202.5, 115.2,  33.7,  83. , 197.7,
        62.3, 102.8, 114.4,  56.9,  26.4,  58.9,  13.9, 195. ,   5.8,
        33.4, 119.8,   1.6, 180.8, 371.1,  20. , 232.9,  42.9, 180.8,
         5.8, 161.3,  51.8, 156.7,  61. , 371.1,  58.9,  53. ,  77.1,
        83.7, 102.7, 148.7,  22.2, 148.7,  45.1, 246.7, 235.2,  88.5,
        99.4, 177.6,

In [49]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test2, predictions4))
print('Mean Squared Error:', metrics.mean_squared_error(y_test2, predictions4))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test2, predictions4)))
print("Training Accuracy = ", model4.score(X_train2, y_train2))
print("Test Accuracy = ", model4.score(X_test2, y_test2))

Mean Absolute Error: 73.76935483870967
Mean Squared Error: 9066.005000000001
Root Mean Squared Error: 95.21557120555441
Training Accuracy =  1.0
Test Accuracy =  -0.11462948265060269


In [50]:
cleaned_df

full_name  Year  PPR_Points  Targets  Rec  Air_Yards  \
0                    Zay Jones  2019        35.0       27   20        234   
1                    Zay Jones  2019        13.9       18    7        250   
2                    Zay Jones  2018       152.1      100   54       1256   
4                  Zach Pascal  2019       133.3       72   41        800   
6                 Willie Snead  2019        95.1       46   31        409   
7                 Willie Snead  2018       134.2       95   62        775   
8                 Willie Snead  2017        17.2       16    8        107   
9                 Willie Snead  2016       185.5      104   72        783   
11                 Will Fuller  2019       134.0       71   49       1011   
12                 Will Fuller  2018       106.3       45   32        623   
13                 Will Fuller  2017       113.2       50   28        794   
23             Tyrell Williams  2019       143.1       64   42        870   
24             Tyrell Williams  2018       134.5       63   40        789   
25             Tyrell Williams  2017       139.8       69   43        988   
26             Tyrell Williams  2016       216.9      119   69       1382   
28                 Tyreek Hill  2019       188.3       89   58       1149   
29                 Tyreek Hill  2018       294.0      132   82       1898   
30                 Tyreek Hill  2017       241.2      105   75       1227   
32               Tyler Lockett  2019       235.2      110   82       1364   
33               Tyler Lockett  2018       217.1       67   55        910   
34               Tyler Lockett  2017       118.3       71   45        922   
35               Tyler Lockett  2016       124.1       66   41        698   
37                  Tyler Boyd  2019       226.9      148   90       1186   
38                  Tyler Boyd  2018       221.1      108   76       1087   
39                  Tyler Boyd  2017        56.5       32   22        218   
41                  Trey Quinn  2019        51.8       47   26        324   
43                Trevor Davis  2019        28.6       10    7         82   
47                Trevor Davis  2017        13.3        7    5         69   
49                Trent Taylor  2018        53.5       40   26        254   
51             Trent Sherfield  2019        12.0       13    4        195   
53              Tre'Quan Smith  2019        71.4       25   18        226   
56             Travis Benjamin  2019         9.0       16    6        214   
57             Travis Benjamin  2018        39.6       23   11        526   
58             Travis Benjamin  2017       124.3       65   34       1012   
59             Travis Benjamin  2016       138.4       75   47       1038   
60             Travis Benjamin  2015       195.8      125   68       1438   
61             Travis Benjamin  2014        68.5       46   18        855   
62             Travis Benjamin  2013        20.0       13    5        243   
64                Torrey Smith  2017        90.7       68   36        877   
65                Torrey Smith  2016        64.7       49   20        660   
66                Torrey Smith  2015       123.3       62   33        869   
67                Torrey Smith  2014       191.7       92   49       1358   
68                Torrey Smith  2013       201.8      137   65       2122   
69                Torrey Smith  2012       183.4      110   49       2002   
71              Tommylee Lewis  2017        29.0       15   10        152   
73                 Tim Patrick  2019        37.8       31   16        315   
79           Terrance Williams  2017       111.3       78   53        663   
80           Terrance Williams  2016       127.4       61   44        716   
81           Terrance Williams  2015       154.0       93   52       1268   
82           Terrance Williams  2014       149.3       66   38       1020   
84                    Ted Ginn  2019        85.9       56   30        891   
85               

In [51]:
testDF2 = cleaned_df.loc[cleaned_df['Targets']>=40]
testDF2 = testDF2.loc[testDF2['PY_Tgt']>=40]
testDF2 = testDF2[['PY_Tgt', 
                     'PY_Rec', 
                     'PY_Yds', 
                     'PY_AY', 
                     'PY_aDOT', 
                     'PY_YAC', 
                     'PY_TD', 
                     'PY_RACR', 
                     'PY_MS_AY', 
                     'PY_TGT_Share',
                     'PY_WOPR',
                     'PY_UAY',
                     'PY_Catch_Rate',
                     'PY_Yds/TGT',
                     'PY_YPR',
                     'PY_AY/Tgt',
                     'PY_AY/Rec',
                     'PY_UAY/Tgt',
                     'PY_UAY/Rec',
                     'PY_YAC/Tgt',
                     'PY_YAC/Rec',
                     'PY_PPR_Points',                   
                     'PPR_Points']]
testDF2

PY_Tgt  PY_Rec  PY_Yds   PY_AY  PY_aDOT  PY_YAC  PY_TD  PY_RACR  \
2       74.0    27.0   316.0   994.0    13.43    63.0    2.0     0.32   
4       44.0    26.0   272.0   470.0    10.68    71.0    2.0     0.58   
6       95.0    62.0   649.0   775.0     8.16   276.0    1.0     0.84   
9      102.0    70.0   990.0  1032.0    10.12   344.0    3.0     0.96   
11      45.0    32.0   503.0   623.0    13.84   166.0    4.0     0.81   
12      50.0    28.0   423.0   794.0    15.88    99.0    7.0     0.53   
13      92.0    47.0   635.0  1428.0    15.52   238.0    2.0     0.44   
23      63.0    40.0   631.0   789.0    12.52   183.0    5.0     0.80   
24      69.0    43.0   728.0   988.0    14.32   320.0    4.0     0.74   
25     119.0    69.0  1059.0  1382.0    11.61   433.0    7.0     0.77   
28     132.0    82.0  1336.0  1898.0    14.38   520.0   10.0     0.70   
29     105.0    75.0  1183.0  1227.0    11.69   451.0    7.0     0.96   
30      83.0    61.0   593.0   662.0     7.98   268.0    6.0     0.90   
32      67.0    55.0   970.0   910.0    13.58   227.0   10.0     1.07   
33      71.0    45.0   555.0   922.0    12.99   188.0    2.0     0.60   
34      66.0    41.0   597.0   698.0    10.58   216.0    1.0     0.86   
35      69.0    51.0   664.0   923.0    13.38   249.0    6.0     0.72   
37     108.0    76.0  1028.0  1087.0    10.06   426.0    7.0     0.95   
49      60.0    43.0   430.0   426.0     7.10   190.0    2.0     1.01   
58      75.0    47.0   677.0  1038.0    13.84   246.0    4.0     0.65   
59     125.0    68.0   966.0  1438.0    11.50   354.0    5.0     0.67   
60      46.0    18.0   314.0   855.0    18.59    43.0    3.0     0.37   
64      49.0    20.0   267.0   660.0    13.47    61.0    3.0     0.40   
65      62.0    33.0   663.0   869.0    14.02   213.0    4.0     0.76   
66      92.0    49.0   767.0  1358.0    14.76   169.0   11.0     0.56   
67     137.0    65.0  1128.0  2122.0    15.49   359.0    4.0     0.53   
68     110.0    49.0   855.0  2002.0    18.20   211.0    8.0     0.43   
69      95.0    50.0   841.0  1692.0    17.81   238.0    7.0     0.50   
79      61.0    44.0   594.0   716.0    11.74   171.0    4.0     0.83   
80      93.0    52.0   840.0  1268.0    13.63   275.0    3.0     0.66   
81      66.0    38.0   633.0  1020.0    15.45   132.0    8.0     0.62   
82      74.0    44.0   736.0   997.0    13.47   204.0    5.0     0.74   
86      95.0    54.0   752.0  1169.0    12.31   182.0    4.0     0.64   
87      97.0    44.0   739.0  1569.0    16.18   241.0   10.0     0.47   
98      91.0    66.0   689.0  1049.0    11.53   236.0    2.0     0.66   
99      51.0    33.0   378.0   559.0    10.96   217.0    1.0     0.68   
100     51.0    36.0   588.0   532.0    10.43   284.0    6.0     1.11   
101     48.0    28.0   241.0   360.0     7.50   108.0    0.0     0.67   
102     74.0    38.0   633.0  1006.0    13.59   261.0    1.0     0.63   
107     87.0    52.0   473.0   636.0     7.31   371.0    5.0     0.74   
108     45.0    32.0   239.0   183.0     4.07   161.0    0.0     1.31   
109     69.0    40.0   418.0   479.0     6.94   229.0    4.0     0.87   
117     83.0    41.0   522.0  1089.0    13.12    90.0    2.0     0.48   
119    118.0    74.0  1248.0  1310.0    11.10   459.0    5.0     0.95   
120    109.0    57.0   966.0  1400.0    12.84   304.0    4.0     0.69   
121    155.0    91.0  1448.0  1898.0    12.25   342.0    6.0     0.76   
122    134.0    69.0  1124.0  1733.0    12.93   381.0    5.0     0.65   
123    131.0    82.0  1345.0  1807.0    13.79   366.0    7.0     0.74   
124    139.0    83.0  1086.0  1622.0    11.67   394.0    5.0     0.67   
125     90.0    50.0   861.0  1115.0    12.39   378.0    7.0     0.77   
132    106.0    65.0   862.0  1107.0    10.44   302.0    4.0     0.78   
133     84.0    59.0   731.0   781.0     9.30   295.0    2.0     0.94   
134    105.0    65.0   683.0   972.0     9.26   257.0    8.0     0.70   
136    148.0   102.0  1021.0  1266.0     8.55   453.0    9.0    

In [52]:
print(len(testDF2['PY_Tgt']))

425


Model #5: Random Forest w/ minimum # of targets

In [53]:
# Create X and y for initial ML model

X3 =  testDF.drop(columns=['PPR_Points'])

model5 = RandomForestRegressor(n_estimators=250)


X_train3, X_test3, y_train3, y_test3 = train_test_split(X3, y, random_state=42)

# Fit RandomFoirest model to the training data


model5.fit(X_train, y_train)

C:\Users\Joseph\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  del sys.path[0]


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=250,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [54]:
# Calculate feature importance
importances = model5.feature_importances_
importances

array([0.03390637, 0.0410626 , 0.12845891, 0.02782207, 0.01284792,
       0.0601353 , 0.01743608, 0.01926761, 0.02861967, 0.04512352,
       0.2171314 , 0.02190338, 0.03295244, 0.04064688, 0.02931382,
       0.01082487, 0.01560498, 0.01869789, 0.01547267, 0.03118089,
       0.02968915, 0.1219016 ])

In [55]:
# Sort the features by their importance
sorted(zip(model5.feature_importances_, X3.columns), reverse=True)

[(0.2171313951483428, 'PY_WOPR'),
 (0.12845890788256503, 'PY_Yds'),
 (0.12190159980142173, 'PY_PPR_Points'),
 (0.060135300851633926, 'PY_YAC'),
 (0.04512351657975264, 'PY_TGT_Share'),
 (0.04106259834793333, 'PY_Rec'),
 (0.04064688183611078, 'PY_Yds/TGT'),
 (0.033906369996641726, 'PY_Tgt'),
 (0.032952440738303536, 'PY_Catch_Rate'),
 (0.031180887664483407, 'PY_YAC/Tgt'),
 (0.02968915381424534, 'PY_YAC/Rec'),
 (0.02931381699411476, 'PY_YPR'),
 (0.02861966722278102, 'PY_MS_AY'),
 (0.027822073919151275, 'PY_AY'),
 (0.021903380924842834, 'PY_UAY'),
 (0.01926761066230184, 'PY_RACR'),
 (0.01869788883613277, 'PY_UAY/Tgt'),
 (0.017436076737547154, 'PY_TD'),
 (0.015604976850825899, 'PY_AY/Rec'),
 (0.015472665700391557, 'PY_UAY/Rec'),
 (0.012847920104855541, 'PY_aDOT'),
 (0.010824869385621196, 'PY_AY/Tgt')]

In [56]:
# Make predictions using a 2nd model


predictions5 = model5.predict(X_test3)
predictions5

array([126.6596, 104.386 , 126.186 , 107.1524,  72.02  ,  55.6496,
        67.0648, 106.7632, 249.9756, 194.51  ,  86.4236,  59.6132,
       236.0388,  55.6492,  89.5556, 146.8108, 222.2976,  74.1704,
       186.7556, 121.3472, 144.092 , 219.538 , 184.4368,  87.102 ,
       235.3396, 200.936 , 121.8588, 193.6612, 200.9096,  66.3064,
       205.7412,  44.5796, 120.4248,  27.8064, 170.1548,  83.3108,
       176.9968, 118.5192, 225.4344, 141.5352, 155.8684,  75.1592,
       240.494 ,  38.5948, 149.6404, 225.5008, 163.584 , 145.8564,
       215.302 , 109.2984, 202.256 , 148.9008,  36.0516, 126.1212,
       309.8324,  35.7196,  96.6224, 113.2744, 223.34  , 130.1792,
       137.8496,  58.5264, 148.5804, 271.4728, 155.4816,  75.8552,
       154.0388, 109.1464,  54.5428, 159.34  , 150.0332,  37.772 ,
        60.772 ,  81.9128,  60.9088,  65.502 , 157.9852,  35.8764,
        72.9068, 130.972 ,  59.38  , 300.1416, 136.4356,  79.8084,
        59.3636, 191.8072,  80.8224,  56.3888, 212.6672, 152.6

In [57]:
print('Model1 Accuracy Tests:')
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, predictions))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, predictions))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))
print("Training Accuracy = ", model.score(X_train, y_train))
print("Test Accuracy = ", model.score(X_test, y_test))
print('---------------------------------------------------------')
print('Model2 Accuracy Tests:')
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test2, predictions2))
print('Mean Squared Error:', metrics.mean_squared_error(y_test2, predictions2))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test2, predictions2)))
print("Training Accuracy = ", model2.score(X_train2, y_train2))
print("Test Accuracy = ", model2.score(X_test2, y_test2))
print('---------------------------------------------------------')
print('Model3 Accuracy Tests:')
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, predictions3))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, predictions3))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, predictions3)))
print("Training Accuracy = ", model3.score(X_train, y_train))
print("Test Accuracy = ", model3.score(X_test, y_test))
print('---------------------------------------------------------')
print('Model4 Accuracy Tests:')
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test2, predictions4))
print('Mean Squared Error:', metrics.mean_squared_error(y_test2, predictions4))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test2, predictions4)))
print("Training Accuracy = ", model4.score(X_train2, y_train2))
print("Test Accuracy = ", model4.score(X_test2, y_test2))
print('---------------------------------------------------------')
print('Model5 Accuracy Tests:')
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, predictions5))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, predictions5))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, predictions5)))
print("Training Accuracy = ", model5.score(X_train3, y_train))
print("Test Accuracy = ", model5.score(X_test3, y_test))
print('---------------------------------------------------------')

Model1 Accuracy Tests:
Mean Absolute Error: 58.579853763440866
Mean Squared Error: 5642.706422577203
Root Mean Squared Error: 75.11795006905608
Training Accuracy =  0.9192231981921242
Test Accuracy =  0.3062515473412535
---------------------------------------------------------
Model2 Accuracy Tests:
Mean Absolute Error: 58.687565591397856
Mean Squared Error: 5715.7000706443005
Root Mean Squared Error: 75.6022491110172
Training Accuracy =  0.9169537570654882
Test Accuracy =  0.2972772668084488
---------------------------------------------------------
Model3 Accuracy Tests:
Mean Absolute Error: 75.552688172043
Mean Squared Error: 9619.690430107526
Root Mean Squared Error: 98.08002054499951
Training Accuracy =  1.0
Test Accuracy =  -0.18270291791916105
---------------------------------------------------------
Model4 Accuracy Tests:
Mean Absolute Error: 73.76935483870967
Mean Squared Error: 9066.005000000001
Root Mean Squared Error: 95.21557120555441
Training Accuracy =  1.0
Test Accuracy 